In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

%cd ~/AI/MSAI
%pwd
print("=" * 70)
print("PREPARING MODEL FOR GGUF CONVERSION")
print("=" * 70)

# Step 1: Load base model in FULL precision (no quantization)
print("\n[1/4] Loading base model in full precision...")
base_model_name = "Qwen/Qwen2.5-1.5B-Instruct"

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,  # Use float16, NOT bitsandbytes
    device_map="auto",
    low_cpu_mem_usage=True
)

tokenizer = AutoTokenizer.from_pretrained(base_model_name)
print("✓ Base model loaded")

# Step 2: Load LoRA adapter
print("\n[2/4] Loading LoRA adapter...")
lora_path = "./oral_exam_model_final"

model = PeftModel.from_pretrained(
    base_model,
    lora_path,
    torch_dtype=torch.float16
)
print("✓ LoRA adapter loaded")

# Step 3: Merge and unload
print("\n[3/4] Merging LoRA weights...")
merged_model = model.merge_and_unload()
print("✓ Weights merged")

# Step 4: Save in standard format (no bitsandbytes)
print("\n[4/4] Saving model for GGUF conversion...")
output_dir = "./oral_exam_model_for_gguf"

merged_model.save_pretrained(
    output_dir,
    safe_serialization=True,
    max_shard_size="2GB"
)

tokenizer.save_pretrained(output_dir)

print(f"✓ Model saved to: {output_dir}")
print("\n" + "=" * 70)
print("✓ READY FOR GGUF CONVERSION!")
print("=" * 70)
print("\nNext step:")
print(f"cd ~/AI/MSAI/llama.cpp/llama.cpp")
print(f"python convert_hf_to_gguf.py {output_dir} --outfile oral-exam-professor.gguf --outtype q4_K_M")


/home/sahammond/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/home/sahammond/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/sahammond/AI/MSAI
PREPARING MODEL FOR GGUF CONVERSION

[1/4] Loading base model in full precision...
✓ Base model loaded

[2/4] Loading LoRA adapter...


/home/sahammond/.local/lib/python3.10/site-packages/accelerate/utils/modeling.py:1566: UserWarning: Current model requires 256 bytes of buffer for offloaded layers, which seems does not fit any GPU's remaining memory. If you are experiencing a OOM later, please consider using offload_buffers=True.
  warnings.warn(


✓ LoRA adapter loaded

[3/4] Merging LoRA weights...
✓ Weights merged

[4/4] Saving model for GGUF conversion...
✓ Model saved to: ./oral_exam_model_for_gguf

✓ READY FOR GGUF CONVERSION!

Next step:
cd ~/AI/MSAI/llama.cpp/llama.cpp
python convert_hf_to_gguf.py ./oral_exam_model_for_gguf --outfile oral-exam-professor.gguf --outtype q4_K_M


In [18]:
# Install llama.cpp conversion tools
#%pip install llama-cpp-python 

# Clone llama.cpp repository
#!git clone https://github.com/ggerganov/llama.cpp
%cd llama.cpp

# Convert your model to GGUF
!pwd
!python3 convert_hf_to_gguf.py ~/AI/MSAI/oral_exam_model_for_gguf \
    --outfile oral-exam-professor.gguf \
    --outtype auto  # 4-bit quantization

/home/sahammond/AI/MSAI/llama.cpp
/home/sahammond/AI/MSAI/llama.cpp


/home/sahammond/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


INFO:hf-to-gguf:Loading model: oral_exam_model_for_gguf
INFO:hf-to-gguf:Model architecture: Qwen2ForCausalLM
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: indexing model part 'model-00001-of-00002.safetensors'
INFO:hf-to-gguf:gguf: indexing model part 'model-00002-of-00002.safetensors'
INFO:hf-to-gguf:choosing --outtype f16 from first tensor type (torch.float16)
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:token_embd.weight,         torch.float16 --> F16, shape = {1536, 151936}
INFO:hf-to-gguf:blk.0.attn_norm.weight,    torch.float16 --> F32, shape = {1536}
INFO:hf-to-gguf:blk.0.ffn_down.weight,     torch.float16 --> F16, shape = {8960, 1536}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,     torch.float16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_up.weight,       torch.float16 --> F16, shape = {1536, 8960}
INFO:hf-to-gguf:blk.0.ffn_norm.weight,   